In [ ]:
pip install torch_geometric

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import itertools
import ast
import pandas as pd
import numpy as np
import torch
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/temp_manu.csv').drop(columns=['Unnamed: 0'])
df

,GlobalId,Web,Name,Employees,BusinessDescription,DataSource,ContractManufacturer,ProductManufacturer,Industry,Certification,...,City,County,State,Zip,Latitude,Longitude,Phone,Fax,NAICSCode,NAICSDescription
0,['westindiesvalve_dhs'],['www.westindiesvalve.com'],['WEST INDIES VALVE AND MACHINE WORK CORPORATI...,[0.0],[],['DHS'],['No'],['No'],[],['ASME Certificate'],...,['Penuelas'],['Penuelas'],['PR'],[],[18.0054364],[-66.7276429],['(787) 836-1707'],['NOT AVAILABLE'],[332919.0],['OTHER METAL VALVE AND PIPE FITTING MANUFACTU...
1,['101pipe_dhs'],['www.101pipe.com'],"['101 VERTICAL FABRICATION, INC.']",[50.0],[],['DHS'],['No'],['Yes'],[],[],...,['Fontana'],['San Bernardino'],['California'],[],[34.0677902],[-117.4708951],['(909) 428-6000'],['(909) 428-4535'],[332996.0],['FABRICATED PIPE AND PIPE FITTING MANUFACTURI...
2,['149401-us_mskg'],['www.149401-us.all.biz'],['149401-US'],[],[],['MSKG'],['No'],['Yes'],['Transportation Industry'],['ISO 9001 Certificate'],...,[],[],[],[],[],[],[],[],[],[]
3,['1776fabrication_mskg'],['www.1776fabrication.com'],['1776 Fabrication'],[],[],['MSKG'],['Yes'],['Yes'],['Automotive Industry'],[],...,[],[],[],[],[],[],[],[],[],[]
4,['1dietech_mskg'],['www.1dietech.com'],['1Dietech.'],[],[],['MSKG'],['Yes'],['Yes'],"['Automotive Industry', 'Furniture Industry']",['IATF 16949 Certificate'],...,['Carrollton'],[],['Georgia'],[],[33.566767],[-85.078863],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14741,['windigosigns_d&b'],['www.windigosigns.com'],['windigosigns'],[],"['Windigo Signs, Inc. is primarily engaged in ...",['D&B'],['No'],['Yes'],['Aluminum'],[],...,['Tempe'],['Maricopa'],['Arizona'],[],[],[],['1-844-946-3446'],[],[339950.0],['Sign Manufacturing']
14742,['xantusb_d&b'],['www.xantusb.openfos.com'],['xantusb.openfos'],[],['Xantus Business Systems Llc is primarily eng...,['D&B'],['No'],['Yes'],[],[],...,['Sierra Vista'],['Cochise'],['Arizona'],[],[],[],['1-520-459-7809'],[],[323111.0],['Commercial Printing (except Screen and Books)']
14743,['vicivisionamerica_d&b'],['www.vicivisionamerica.com'],['vicivisionamerica'],[],['Vicivision America Llc is primarily engaged ...,['D&B'],['No'],['Yes'],[],[],...,['Scottsdale'],['Maricopa'],['Arizona'],[],[],[],['1-480-212-5172'],[],[333310.0],['Commercial and Service Industry Machinery Ma...
14744,['infineon_d&b'],['www-blue.infineon.com'],['www-blue.infineon'],[],['Infineon Technologies Americas Corp. is prim...,['D&B'],[],[],"['Concrete', 'Lead', 'Titanium']",[],...,"['Mesa', 'Chandler']",['Maricopa'],['Arizona'],[],[],[],"['1-480-655-7701', '1-480-627-5700']",[],[334413.0],['Semiconductor and Related Device Manufacturi...


In [ ]:
df = df[['GlobalId', 'ProcessCapability']]
df

,GlobalId,ProcessCapability
0,['westindiesvalve_dhs'],[]
1,['101pipe_dhs'],[]
2,['149401-us_mskg'],[]
3,['1776fabrication_mskg'],"['Forming Capability', 'Machining Capability',..."
4,['1dietech_mskg'],"['Forming Capability', 'Laser Cutting Capabili..."
...,...,...
14741,['windigosigns_d&b'],[]
14742,['xantusb_d&b'],[]
14743,['vicivisionamerica_d&b'],[]
14744,['infineon_d&b'],[]


In [ ]:
df['GlobalId'].value_counts()

,count
GlobalId,
['net_az'],10
['us_az'],4
['us_d&b'],4
['shop_d&b'],3
['hub_d&b'],3
...,...
['inland-mfg_mskg'],1
['inlandbuildings_dhs'],1
['inlandlakes_dhs'],1


Cleans up the ProcessCapability column by turning the text '[]' into missing values and parsing all other stringified Python literals into real Python data structures

In [ ]:
df['ProcessCapability'] = df['ProcessCapability'].apply(lambda x:None if x=='[]' else ast.literal_eval(x))

<ipython-input-9-5f50d4be1623>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ProcessCapability'] = df['ProcessCapability'].apply(lambda x:None if x=='[]' else ast.literal_eval(x))


In [ ]:
sum(df['ProcessCapability'].isna())

8090

In [ ]:
df.dropna(subset=['ProcessCapability'], inplace=True)
sum(df['ProcessCapability'].isna())

<ipython-input-11-c6a827cafaf7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['ProcessCapability'], inplace=True)


0

In [ ]:
df['ProcessCapability']

,ProcessCapability
3,"[Forming Capability, Machining Capability, Fin..."
4,"[Forming Capability, Laser Cutting Capability,..."
10,"[Wire EDM Capability, Boring Capability, Casti..."
11,"[Grinding Capability, Etching Capability, Bori..."
13,"[Additive Manufacturing Capability, Casting Ca..."
...,...
14723,"[Assembly Automated, Assembly Non-Automated, T..."
14724,"[Prototyping Validation, Assembly Downstream, ..."
14739,"[Welding Robotic, Welding Manual, Assembly Mec..."
14740,"[Co-Packing, Purify, Treat]"


In [ ]:
df['GlobalId']

,GlobalId
3,['1776fabrication_mskg']
4,['1dietech_mskg']
10,['3dimensional_mskg']
11,['3dmach_mskg']
13,['3dsystems_mskg']
...,...
14723,['gillman_d&b']
14724,['accudyn_d&b']
14739,['wildeck_d&b']
14740,['whitewaternow_d&b']


We flatten each row’s list of `ProcessCapability` values into a two-column edge list—one `(GlobalId, capability)` pair per entry—then packages the pairs into a new DataFrame `edge_df`.

In [ ]:
edges = []
for idx, row in df.iterrows():
    gid = row['GlobalId']
    capabilities = row['ProcessCapability']
    for cap in capabilities:
        edges.append((gid, cap))

edge_df = pd.DataFrame(edges, columns=['GlobalId', 'ProcessCapability'])

In [ ]:
edge_df

,GlobalId,ProcessCapability
0,['1776fabrication_mskg'],Forming Capability
1,['1776fabrication_mskg'],Machining Capability
2,['1776fabrication_mskg'],Finishing Capability
3,['1776fabrication_mskg'],Fabricating Capability
4,['1776fabrication_mskg'],CNC Machining Capability
...,...,...
70362,['westfall-technik_d&b'],Prototyping Low-Volume
70363,['westfall-technik_d&b'],Prototyping High-Volume
70364,['westfall-technik_d&b'],Assembly Kitting
70365,['westfall-technik_d&b'],Automation Cell Engineering


We implement labelencoding for the dataframe

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le_gid = LabelEncoder()
le_cap = LabelEncoder()

edge_df['gid_encoded'] = le_gid.fit_transform(edge_df['GlobalId'])
edge_df['cap_encoded'] = le_cap.fit_transform(edge_df['ProcessCapability'])
edge_df['cap_encoded'] += edge_df['gid_encoded'].max() + 1      #offset capability code so that node IDs in the graph are unique
edge_df

,GlobalId,ProcessCapability,gid_encoded,cap_encoded
0,['1776fabrication_mskg'],Forming Capability,0,9137
1,['1776fabrication_mskg'],Machining Capability,0,10014
2,['1776fabrication_mskg'],Finishing Capability,0,9035
3,['1776fabrication_mskg'],Fabricating Capability,0,8949
4,['1776fabrication_mskg'],CNC Machining Capability,0,7337
...,...,...,...,...
70362,['westfall-technik_d&b'],Prototyping Low-Volume,6475,11358
70363,['westfall-technik_d&b'],Prototyping High-Volume,6475,11355
70364,['westfall-technik_d&b'],Assembly Kitting,6475,6945
70365,['westfall-technik_d&b'],Automation Cell Engineering,6475,7052


Link-prediction (or edge classification) is essentially a binary-classification problem. Positive edges tell the model what a real connection looks like in terms of node features and graph context. Negative edges give counter-examples, preventing the model from simply predicting “edge” for every pair.

In this code snippet, we shuffle known part-capability links, split them 80/20 into training and test positives, then construct matching negative edges—balanced in the test set and smaller in the training set—to create datasets ready for training and evaluating a link-prediction model.

In binary-classification tasks, edges that truly exist (the positive edges) are often vastly out-numbered by negatives (non-edges). If we feed this raw distribution to a model, it can achieve deceptively high accuracy simply by predicting the majority class every time (“no edge here”), yet it will perform poorly on the minority class that we are actually concerned with. Thus we sub-sample the majority class so that the model sees a near mix of samples and is not biased.

In [ ]:
#we create a list of positive edges (existing links) by pairing each encoded part-ID (gid_encoded) with each encoded capability (cap_encoded).
pos_edges = list(zip(edge_df['gid_encoded'], edge_df['cap_encoded']))

#we shuffle the edges in order to reduce biasing
random.shuffle(pos_edges)

#we split it into an 80:20 training and testing set
split = int(len(pos_edges) * 0.8)
train_edges = pos_edges[:split]
test_pos_edges = pos_edges[split:]

#collect the unique sets of gids and capabailities
all_gids = edge_df['gid_encoded'].unique()
all_caps = edge_df['cap_encoded'].unique()

#we generate all possible part-capability pairs (Cartesian product) and remove the ones that actually exist; what remains are candidate negative edges—pairs that are not present in the data.
possible_neg_edges = list(set((g, c) for g in all_gids for c in all_caps) - set(pos_edges))
test_neg_edges = random.sample(possible_neg_edges, len(test_pos_edges))   #randomly select a negative edge equal in size to a positive edge
possible_neg_set = set(possible_neg_edges)
test_neg_set = set(test_neg_edges)

#we remove the test negatives from the overall negative pool, then sample a training-negative set that is half the size of the training-positive set
train_neg_pool = possible_neg_set - test_neg_set
train_neg_edges = random.sample(list(train_neg_pool), len(train_edges) // 2)

In [ ]:
len(train_edges), len(train_neg_edges), len(test_pos_edges), len(test_neg_edges)

(56293, 28146, 14074, 14074)

We convert the raw edge lists into [2,E] tensors (for pytorch to process), create one-hot node features, and build a PyG Data object whose edge_index contains both directions of every training edge, giving an undirected graph that is ready for Graph Neural Network training and later link-prediction evaluation.

In [ ]:
#we turn the list of (source, target) tuples into a tensor of integers
def encode_edges(edge_list):
    return torch.tensor(edge_list, dtype=torch.long).T  # shape [2, num_edges]

#create tensors for every edge
train_edge_index = encode_edges(train_edges)
test_pos_index = encode_edges(test_pos_edges)
test_neg_index = encode_edges(test_neg_edges)
train_neg_index = encode_edges(train_neg_edges)

num_nodes = max(edge_df['cap_encoded'].max(), edge_df['gid_encoded'].max()) + 1`  #total number of nodes
x = torch.eye(num_nodes)

#we assemble the training graph
data = Data(x=x, edge_index=torch.cat([train_edge_index, train_edge_index.flip(0)], dim=1))

In [ ]:
data

Data(x=[13147, 13147], edge_index=[2, 112586])

In [ ]:
train_edges[:5]

[(3621, 9726), (4886, 11813), (397, 9137), (6330, 9256), (3862, 9256)]

In [ ]:
train_edge_index

tensor([[ 3621,  4886,   397,  ...,  3583,  4381,  4409],
        [ 9726, 11813,  9137,  ...,  9128, 12062,  9821]])

In [ ]:
train_edge_index.shape

torch.Size([2, 56293])

In [ ]:
x

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [ ]:
num_nodes

13147

In [ ]:
edge_df['cap_encoded'].max()

13146

In [ ]:
edge_df['gid_encoded'].max()

6645

In [ ]:
torch.cat([train_edge_index, train_edge_index.flip(0)])

tensor([[ 3621,  4886,   397,  ...,  3583,  4381,  4409],
        [ 9726, 11813,  9137,  ...,  9128, 12062,  9821],
        [ 9726, 11813,  9137,  ...,  9128, 12062,  9821],
        [ 3621,  4886,   397,  ...,  3583,  4381,  4409]])

In [ ]:
torch.cat([train_edge_index, train_edge_index.flip(0)]).shape

torch.Size([4, 56293])

In [ ]:
x[13146]

tensor([0., 0., 0.,  ..., 0., 0., 1.])

The GCNEncoder is a compact, two-layer Graph Convolutional Network (GCN) that turns raw node features into latent embeddings.

In [ ]:
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

We instantiate a two-layer GCN to turn node features into 64-dimensional embeddings. Then, we introduce a dot-product `decode` function that scores any pair of nodes, and `BCEWithLogitsLoss` with Adam (lr = 0.01), which is set up to teach the embeddings to give high scores to real edges and low scores to sampled negatives.  
Together, these three components form a complete, trainable link-prediction pipeline: **encoder --> decoder --> loss/optimizer**.

In [ ]:
def decode(z, edge_index):
    return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)

model = GCNEncoder(in_channels=x.size(1), hidden_channels=64) #instantiate the GCNEncoder, where the model is a 2 layer convolutional network
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) #Adam combines momentum and adaptive learning-rates
loss_fn = torch.nn.BCEWithLogitsLoss()  #it first applies a sigmoid to each score, then computes BCE in a numerically-stable way`

For each of 20 epochs we produce node embeddings with a two-layer GCN, score real and sampled-negative edges via a dot-product decoder, and concatenate those logits with matching 1/0 labels.
BCEWithLogitsLoss computes the error, gradients are back-propagated, and Adam adjusts the GCN weights.
The printed loss shows whether the model is learning to separate existing edges from non-edges over time.

In [ ]:
for epoch in tqdm(range(1, 21)):    #loops 20 times
    model.train()   #we train the model
    optimizer.zero_grad()   #we clear leftover gradients

    z = model(data.x, data.edge_index)    #we run a forward pass
    pos_out = decode(z, train_edge_index)   #we compute dot product for positive and negative training edges
    neg_out = decode(z, train_neg_index)

    out = torch.cat([pos_out, neg_out])   #we join the tensors

    y_true = torch.cat([                  #we build a matching label vector: 1’s for real edges, 0’s for sampled non-edges
        torch.ones(pos_out.size(0)),
        torch.zeros(neg_out.size(0))
    ])

    loss = loss_fn(out, y_true)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch} | Loss: {loss.item():.4f}")


  5%|▌         | 1/20 [00:01<00:26,  1.38s/it]

Epoch 1 | Loss: 0.6931


 10%|█         | 2/20 [00:02<00:22,  1.27s/it]

Epoch 2 | Loss: 0.6242


 15%|█▌        | 3/20 [00:03<00:21,  1.26s/it]

...

Epoch 18 | Loss: 0.2288


 95%|█████████▌| 19/20 [00:26<00:01,  1.52s/it]

Epoch 19 | Loss: 0.2263


100%|██████████| 20/20 [00:27<00:00,  1.39s/it]

Epoch 20 | Loss: 0.2247


In [ ]:
z.shape

torch.Size([13147, 64])

In [ ]:
test_pos_index.shape

torch.Size([2, 14074])

In [ ]:
z[test_pos_index[0]].shape

torch.Size([14074, 64])

In [ ]:
z[test_pos_index[1]].shape

torch.Size([14074, 64])

In [ ]:
test_pos_index[0]

tensor([3668, 4622, 5459,  ...,  376,  843, 2632])

In [ ]:
z[test_pos_index[0]]

tensor([[-0.2798,  0.2321,  0.2523,  ..., -0.2384,  0.2697, -0.1834],
        [-0.3043,  0.2524,  0.2745,  ..., -0.2590,  0.2937, -0.2002],
        [-0.1693,  0.1393,  0.1524,  ..., -0.1448,  0.1655, -0.1100],
        ...,
        [-0.1922,  0.1596,  0.1733,  ..., -0.1638,  0.1859, -0.1249],
        [ 0.0734, -0.0121, -0.1187,  ...,  0.0192, -0.0689,  0.0542],
        [-0.2995,  0.2490,  0.2700,  ..., -0.2552,  0.2888, -0.1968]],
       grad_fn=<IndexBackward0>)

In [ ]:
pos_out.shape, neg_out.shape, out.shape

(torch.Size([56293]), torch.Size([28146]), torch.Size([84439]))

We now prepare the evaluation metrics for the project

In [ ]:
model.eval()
with torch.no_grad():
    z = model(data.x, data.edge_index)
    pos_pred = torch.sigmoid(decode(z, test_pos_index)).numpy()
    neg_pred = torch.sigmoid(decode(z, test_neg_index)).numpy()

y_true = [1]*len(pos_pred) + [0]*len(neg_pred)
y_score = list(pos_pred) + list(neg_pred)

auc = roc_auc_score(y_true, y_score)
ap = average_precision_score(y_true, y_score)
print(f"AUC: {auc:.4f}, Average Precision: {ap:.4f}")

AUC: 0.9679, Average Precision: 0.9739


The loop prints the first 30 test-set positive edges, converting their integer node IDs back into the original GlobalId -> ProcessCapability pairs so we can verify what real relationships the model will be asked to predict.

In [ ]:
for i in range(30):
    u, v = test_pos_edges[i]
    print(le_gid.inverse_transform([u])[0], '-->', le_cap.inverse_transform([v - len(le_gid.classes_)])[0])


['manortool_dhs'] --> Milling Capability
['plastic-bags_mskg'] --> Anodizing Capability
['skiair_dhs'] --> Surface Preparation Capability
['zioninduction_dhs'] --> Hardening Capability
['gain-ind_dhs'] --> Machining Capability
['gcnsupply_mskg'] --> Electro Plating Capability
['exactinc_dhs'] --> Grinding Capability
['swisslineprecision_mskg'] --> Polishing Capability
['generalplug_dhs'] --> Tapping Capability
['outroar_d&b'] --> Validation
['vansindustrial_dhs'] --> Fabricating Capability
['cuevasmachine_mskg'] --> Milling Capability
['starkindustrial_mskg'] --> Drilling Capability
['metalquest_dhs'] --> Machining Capability
['wesco_az'] --> Finishing capability
['stretchformingcorp_dhs'] --> Machining Capability
['dcgprecision_mskg'] --> Forming Capability
['bmprecision_dhs'] --> Milling Capability
['innoflexcorp_dhs'] --> Etching Capability
['techplating_dhs'] --> Electro Plating Capability
['specialtytoolanddie_dhs'] --> Turning Capability
['graphiccomp_d&b'] --> Die Cutting
['cema